In [11]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.linear_model import LogisticRegression as LR
from sklearn.model_selection import train_test_split  # 随机划分数据集
import matplotlib.pyplot as plt
from warnings import simplefilter

simplefilter(action='ignore') #忽略警告

In [12]:
#读取数据

data = pd.read_csv('data/level_2/level_2 (144).csv',encoding = 'gb2312',names=['id', 'frame', 'x', 'y'])  # 读入csv文件中的特征

data.info()#输出基本信息

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2245 entries, 0 to 2244
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      2244 non-null   float64
 1   frame   2245 non-null   object 
 2   x       2245 non-null   object 
 3   y       2245 non-null   object 
dtypes: float64(1), object(3)
memory usage: 70.3+ KB


In [13]:
# pd.set_option('display.max_columns', None) #显示所有列
# pd.set_option('display.max_rows', None)  #显示所有行
pd.DataFrame(data)

,id,frame,x,y
0,NaN,frame,x,y
1,0.0,1,198,242
2,1.0,1,197,252
3,2.0,1,196,261
4,3.0,1,197,270
...,...,...,...,...
2240,63.0,73,262,270
2241,64.0,73,268,274
2242,65.0,73,261,270
2243,66.0,73,257,269


In [14]:
#判断缺失值
(data.isnull()).sum()#判断是否有缺失值

id       1
frame    0
x        0
y        0
dtype: int64

In [ ]:
#观察x的数据分布，发现不存在十分异常的数据
data_x = data.sort_values('x')
print(data_x['x'].describe())
data_x['x'].hist()

count     2245
unique     132
top        235
freq        57
Name: x, dtype: object


<AxesSubplot:>

In [ ]:
#观察y的数据分布，发现不存在十分异常的数据
data_y = data.sort_values('y')
print(data_y['y'].describe())
data_y['y'].hist()

In [ ]:
#眼部
#求最大值
#定义几个要用的变量
left_max=0;
right_max=0;
length=data.shape[0];
for i in range(0,length-1):
    i=i+1;
    id=data.iloc[i]["id"];
    if id == 39.0 :
        x39=float(data.iloc[i]["x"])
        x36=float(data.iloc[i-3]["x"])
        x45=float(data.iloc[i+6]["x"])
        x42=float(data.iloc[i+3]["x"])
        w_left_sub=x39-x36;#左眼的宽度差值
        w_right_sub=x45-x42;#右眼的宽度差值
        y38=float(data.iloc[i-1]["y"])
        y40=float(data.iloc[i+1]["y"])
        y43=float(data.iloc[i+4]["y"])
        y47=float(data.iloc[i+8]["y"])
        h_left_sub=y40-y38;#左眼的高度差值
        h_right_sub=y47-y43;#右眼的高度差值
        
        #求最大值
        left_max_temp=w_left_sub*h_left_sub;
        right_max_temp=w_right_sub*h_right_sub;
        if left_max_temp>left_max:
            left_max=left_max_temp;
        if right_max_temp>right_max:
            right_max=right_max_temp;

In [ ]:
import math

length=data.shape[0];
xl={};#左眼中间横坐标
yl={};#左眼中间纵坐标
xr={};#右眼中间横坐标
yr={};#右眼中间纵坐标
xm={};#上唇中间的横坐标
ym={};#下唇中间横坐标
A={};
B={};
C={};
d={};#上唇中部到两眼连线的距离
head_amplitude={}; #头部摆动幅度

left_max_ratio={};
right_max_ratio={};

K={};

for i in range(0,length-1):
    i=i+1;
    id=data.iloc[i]["id"];
    frame=int(data.iloc[i]["frame"]);
    if id == 39.0 :
        xl[frame]=(float(data.iloc[i-2]["x"])+float(data.iloc[i-1]["x"])+float(data.iloc[i+1]["x"])+float(data.iloc[i+2]["x"]))/4
        yl[frame]=(float(data.iloc[i-2]["y"])+float(data.iloc[i-1]["y"])+float(data.iloc[i+1]["y"])+float(data.iloc[i+2]["y"]))/4
        
        x39=float(data.iloc[i]["x"])
        x36=float(data.iloc[i-3]["x"])
        x45=float(data.iloc[i+6]["x"])
        x42=float(data.iloc[i+3]["x"])
        w_left_sub=x39-x36;#左眼的宽度差值
        w_right_sub=x45-x42;#右眼的宽度差值
        y38=float(data.iloc[i-1]["y"])
        y40=float(data.iloc[i+1]["y"])
        y43=float(data.iloc[i+4]["y"])
        y47=float(data.iloc[i+8]["y"])
        h_left_sub=y40-y38;#左眼的高度差值
        h_right_sub=y47-y43;#右眼的高度差值
        
        #求百分比
        left_max_temp=w_left_sub*h_left_sub;
        left_max_ratio[frame]=left_max_temp/left_max;#左眼目前/左眼最大
        right_max_temp=w_right_sub*h_right_sub;
        right_max_ratio[frame]=right_max_temp/right_max;
        
    if id == 45.0 :
        xr[frame]=(float(data.iloc[i-2]["x"])+float(data.iloc[i-1]["x"])+float(data.iloc[i+1]["x"])+float(data.iloc[i+2]["x"]))/4
        yr[frame]=(float(data.iloc[i-2]["y"])+float(data.iloc[i-1]["y"])+float(data.iloc[i+1]["y"])+float(data.iloc[i+2]["y"]))/4        
    if id == 51.0 :
        xm[frame]=float(data.iloc[i]["x"])
        ym[frame]=float(data.iloc[i]["y"])
#     if id == 54.0:
#         x54=float(data.iloc[i]["x"])
#         x48=float(data.iloc[i-6]["x"])
#         w=x54-x48;#嘴巴的横
#         y50=float(data.iloc[i-4]["y"])
#         y58=float(data.iloc[i+4]["y"])
#         y52=float(data.iloc[i-2]["y"])
#         y56=float(data.iloc[i+2]["y"])
#         h=0.5*(abs(y50-y58)+abs(y52-y56));#嘴巴的纵
#         K[frame]=h/w;
    if id == 60.0 :
        x60=float(data.iloc[i]["x"])
        y60=float(data.iloc[i]["y"])
        x62=float(data.iloc[i+2]["x"])
        y62=float(data.iloc[i+2]["y"])
        x66=float(data.iloc[i+6]["x"])
        y66=float(data.iloc[i+6]["y"])
        #点62到点60距离的平方
        d_62_60_2=pow((x62-x60), 2)+ pow((y62-y60), 2)
        d_66_60_2=pow((x66-x60), 2)+ pow((y66-y60), 2)
        d_66_62_2=pow((x66-x62), 2)+ pow((y66-y62), 2)
        #距离
        d_62_60=pow(d_62_60_2, 0.5)
        d_66_60=pow(d_66_60_2, 0.5)
        d_66_62=pow(d_66_62_2, 0.5)
        MOA=(d_62_60_2 + d_66_60_2 - d_66_62_2) / (2 * d_62_60 * d_66_60)
    
    
dmax=0;
#求上唇中间到双眼连线的距离
for j in yr:
    A[j]=yr[j]-yl[j];
    B[j]=xl[j]-xr[j];
    C[j]=xr[j]*yl[j]-xl[j]*yr[j]
    d[j] = np.abs(A[j] * xm[j] + B[j] * ym[j] + C[j] ) / (np.sqrt(A[j]**2 + B[j]**2))
    dmax=max(dmax,d[j]);#更新最大值
    

# print("最大的d是：",dmax)

#求出头的摆动幅度（转化为度）
for k in yr:

    head_amplitude[k]=math.degrees(math.acos(d[k]/dmax));

In [ ]:
#输出到文件
        
#字典中的key值即为csv中的列名

frame_col={};

for k in range(1,frame+1):
    frame_col[k]=k;
    
df = pd.DataFrame({'frame':frame_col,'head_amplitude':head_amplitude,
                  'left_max_ratio':left_max_ratio,'right_max_ratio':right_max_ratio,
                  'MOA':MOA})
 
#将DataFrame存储为csv，index表识时候显示行名，default=True
df.to_csv("data_new/level_2/2_144.csv",index=False,sep=',')